In [70]:
import seaborn as sns


# Import a bunch of libraries.
import pandas as pd
import re
import string
import nltk
import json
import os

from nltk.stem.porter import PorterStemmer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

## Helper Functions

In [125]:

#Helper Functions
# remove special char
#This function removes , special characters, digits and stopwords from the data 

stopwords = ['the','a','is','am','are', 'i','love','get','go']

def clean_text_data(text_data):
    #Lowercase the reviews
    text_data=text_data.apply(lambda x: x.lower() if type(x) == str else x)
    
     # Removing .COM/BILL 
    text_data=text_data.apply(lambda x: re.sub(r'.com/bill', ' ', x) if type(x) == str else x)
    
      # Removing .COM
    #text_data=text_data.apply(lambda x: re.sub(r'.com', ' com', x) if type(x) == str else x)
    
      # Removing www.
    text_data=text_data.apply(lambda x: re.sub(r'www.', ' ', x) if type(x) == str else x)

    #Remove digits and words containing digits
    text_data=text_data.apply(lambda x: re.sub('\w*\d\w*',' ', x) if type(x) == str else x)

    #Remove Excape Characters
    text_data=text_data.apply(lambda x: re.sub(r'(\\r\\n)+', ' ', x) if type(x) == str else x)
    
    #Remove Punctuations
    text_data=text_data.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x) if type(x) == str else x)
    
    # Removing extra spaces
    text_data=text_data.apply(lambda x: re.sub(' +',' ',x) if type(x) == str else x)
    
    # Removing URLs
    text_data=text_data.apply(lambda x: re.sub(r'http\w*', ' ', x) if type(x) == str else x)
    
   #removing stop word 
    text_data = text_data.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]) if type(x) == str else x )

   
    #Porter Stemmer
    porter = PorterStemmer()
    text_data = [porter.stem(word) for word in text_data]

    return text_data


# read city names:
def readCityNamesFromFile():
    with open('../data/city_names_new.txt') as f:
        lines = [line.rstrip().lower() for line in f]
    return lines
    
    
city_names = readCityNamesFromFile()



## DataSet

In [126]:
df_Data = pd.read_csv("../data/sample-data.csv")
df_Data.head()


,auth_ts,mcc,mid,auth_merch_name,auth_amt,local_amt
0,2021-08-03 05:15:59.000,5812,4445028928044,TST* THE BLUEBERRY MUF PLYMOUTH MA,3.41,3.41
1,2021-08-03 05:15:59.000,5818,160146000762203,Blink amzn.com/bill WA,3.00,3.00
2,2021-08-03 05:16:00.000,5942,235251000762203,AMZN Mktp US Amzn.com/bill WA,31.97,31.97
3,2021-08-03 05:16:00.000,5814,385106000000000,MCDONALD'S F103 ANNAPOLIS MD,8.88,8.88
4,2021-08-03 05:16:00.000,5945,527021000203861,Oculus Menlo Park CA,0.00,0.00


In [127]:
df_Data.describe()

,mcc,auth_amt,local_amt
count,100000.000000,100000.000000,99981.00000
mean,5812.932090,13.746925,33.65583
std,644.894402,31.369086,1696.17650
min,742.000000,0.000000,0.00000
25%,5735.000000,2.770000,2.60000
50%,5814.000000,7.990000,7.99000
75%,5818.000000,14.480000,14.25000
max,9402.000000,3192.000000,279950.00000


### Drop columns that are not needed


In [128]:
# exclude date , mid , amount and local amount columns
#df_Data = df_Data.drop(columns=['auth_ts', 'mcc' ,'mid','auth_amt','local_amt'],axis=1)
#df_Data.head()

### Split train and test data

In [129]:
df_train, df_test = train_test_split(df_Data, test_size=0.20, random_state=42)
df_test.to_csv('../data/df_test.csv', index=False)
df_temp = df_train

### Clean merchant column - repove special char pantuationc and porter stem it

In [130]:
train_mname_1 = df_train["auth_merch_name"].map(str)
mname_train_mname_1_squeeze = train_mname_1.squeeze()
mname_1_train =  clean_text_data(mname_train_mname_1_squeeze) #clean text to remove stop words and especial characters 
mname_1_train_df = pd.DataFrame(mname_1_train, columns=['cleaned_mname_1'])
df_train['cleaned_mname_1'] =  mname_1_train
df_train.head(15)


C:\Users\ashwi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,auth_ts,mcc,mid,auth_merch_name,auth_amt,local_amt,cleaned_mname_1
75220,2021-08-29 08:20:50.000,7999,188418000053360,SQ *PG POOL Mount Rainier MD,1.00,1.00,sq pg pool mount rainier md
48955,2021-08-14 01:41:27.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,12.83,12.83,apple ca
44966,2021-08-14 01:45:28.000,5818,342475000144509,SIE*PLAYSTATIONNETWORK 877-971-7669 CA,4.97,4.97,sie playstationnetwork ca
13568,2021-08-03 06:14:58.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.00,0.00,apple ca
92727,2021-08-29 08:14:41.000,5735,311204598883,APPLE.COM/BILL www.apple.com CA,9.99,9.99,apple apple com ca
51349,2021-09-09 08:48:58.000,5942,784959000762203,Amazon.com Amzn.com/bill WA,2.02,2.02,amazon com amzn wa
86979,2021-08-15 02:43:03.000,5942,784959000762203,AMAZON.COM AMZN.COM/BILL WA,0.00,0.00,amazon com amzn wa
3806,2021-09-09 13:31:35.000,5735,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.99,0.99,apple ca
91822,2021-09-09 06:57:33.000,5735,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.00,0.00,apple ca
6006,2021-08-03 05:38:48.000,5816,145376000144509,PLAYSTATION NETWORK 800-345-7669 CA,7.99,7.99,playstation network ca


In [131]:
# drom merchant name as it is nto needed anymore
#df_train = df_train.drop(columns=['auth_merch_name'],axis=1)
df_train['cleaned_mname_1'] = df_train['cleaned_mname_1'].str.strip()
df_train.head()

C:\Users\ashwi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,auth_ts,mcc,mid,auth_merch_name,auth_amt,local_amt,cleaned_mname_1
75220,2021-08-29 08:20:50.000,7999,188418000053360,SQ *PG POOL Mount Rainier MD,1.00,1.00,sq pg pool mount rainier md
48955,2021-08-14 01:41:27.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,12.83,12.83,apple ca
44966,2021-08-14 01:45:28.000,5818,342475000144509,SIE*PLAYSTATIONNETWORK 877-971-7669 CA,4.97,4.97,sie playstationnetwork ca
13568,2021-08-03 06:14:58.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.00,0.00,apple ca
92727,2021-08-29 08:14:41.000,5735,311204598883,APPLE.COM/BILL www.apple.com CA,9.99,9.99,apple apple com ca


In [132]:
df_train.head()

,auth_ts,mcc,mid,auth_merch_name,auth_amt,local_amt,cleaned_mname_1
75220,2021-08-29 08:20:50.000,7999,188418000053360,SQ *PG POOL Mount Rainier MD,1.00,1.00,sq pg pool mount rainier md
48955,2021-08-14 01:41:27.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,12.83,12.83,apple ca
44966,2021-08-14 01:45:28.000,5818,342475000144509,SIE*PLAYSTATIONNETWORK 877-971-7669 CA,4.97,4.97,sie playstationnetwork ca
13568,2021-08-03 06:14:58.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.00,0.00,apple ca
92727,2021-08-29 08:14:41.000,5735,311204598883,APPLE.COM/BILL www.apple.com CA,9.99,9.99,apple apple com ca


#### Breakdown the column - split by spance-- last stringis state

In [133]:
df_train[['mname', 'state']] = df_train.cleaned_mname_1.apply(
   lambda x: pd.Series(str(x).rsplit(" ", 1)))

df_train.head(10)

,auth_ts,mcc,mid,auth_merch_name,auth_amt,local_amt,cleaned_mname_1,mname,state
75220,2021-08-29 08:20:50.000,7999,188418000053360,SQ *PG POOL Mount Rainier MD,1.00,1.00,sq pg pool mount rainier md,sq pg pool mount rainier,md
48955,2021-08-14 01:41:27.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,12.83,12.83,apple ca,apple,ca
44966,2021-08-14 01:45:28.000,5818,342475000144509,SIE*PLAYSTATIONNETWORK 877-971-7669 CA,4.97,4.97,sie playstationnetwork ca,sie playstationnetwork,ca
13568,2021-08-03 06:14:58.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.00,0.00,apple ca,apple,ca
92727,2021-08-29 08:14:41.000,5735,311204598883,APPLE.COM/BILL www.apple.com CA,9.99,9.99,apple apple com ca,apple apple com,ca
51349,2021-09-09 08:48:58.000,5942,784959000762203,Amazon.com Amzn.com/bill WA,2.02,2.02,amazon com amzn wa,amazon com amzn,wa
86979,2021-08-15 02:43:03.000,5942,784959000762203,AMAZON.COM AMZN.COM/BILL WA,0.00,0.00,amazon com amzn wa,amazon com amzn,wa
3806,2021-09-09 13:31:35.000,5735,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.99,0.99,apple ca,apple,ca
91822,2021-09-09 06:57:33.000,5735,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.00,0.00,apple ca,apple,ca
6006,2021-08-03 05:38:48.000,5816,145376000144509,PLAYSTATION NETWORK 800-345-7669 CA,7.99,7.99,playstation network ca,playstation network,ca


In [134]:
city_names

['alexander city',
 'andalusia',
 'anniston',
 'athens',
 'atmore',
 'auburn',
 'bessemer',
 'birmingham',
 'chickasaw',
 'clanton',
 'cullman',
 'decatur',
 'demopolis',
 'dothan',
 'enterprise',
 'eufaula',
 'florence',
 'fort payne',
 'gadsden',
 'greenville',
 'guntersville',
 'huntsville',
 'jasper',
 'marion',
 'mobile',
 'montgomery',
 'opelika',
 'ozark',
 'phenix city',
 'prichard',
 'scottsboro',
 'selma',
 'sheffield',
 'sylacauga',
 'talladega',
 'troy',
 'tuscaloosa',
 'tuscumbia',
 'tuskegee',
 'alaska',
 'anchorage',
 'cordova',
 'fairbanks',
 'haines',
 'homer',
 'juneau',
 'ketchikan',
 'kodiak',
 'kotzebue',
 'nome',
 'palmer',
 'seward',
 'sitka',
 'skagway',
 'valdez',
 'arizona',
 'ajo',
 'avondale',
 'bisbee',
 'casa grande',
 'chandler',
 'clifton',
 'douglas',
 'flagstaff',
 'florence',
 'gila bend',
 'glendale',
 'globe',
 'kingman',
 'lake havasu city',
 'mesa',
 'nogales',
 'oraibi',
 'phoenix',
 'prescott',
 'scottsdale',
 'sierra vista',
 'tempe',
 'tombsto

In [135]:
df_train['city'] = ''  

for index, row in df_train.iterrows():  
    str1 = row['mname'].split(" ")[-1:][0]
    str2 = " ".join(row['mname'].split(" ")[-2:])   
    if str2 in city_names:   
        df_train.loc[index,'city'] = str2
        df_train.loc[index,'mname'] = row['mname'].replace(str2,'')
    elif str1 in city_names:
        df_train.loc[index,'city']  = str1
        df_train.loc[index,'mname'] = row['mname'].replace(str1,'')   
        
    


C:\Users\ashwi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [136]:
df_train.city.unique()

array(['', 'bossier city', 'indianapolis', ..., 'plattsmouth', 'dubuque',
       'guilford'], dtype=object)

In [137]:
df_train.head(100)

,auth_ts,mcc,mid,auth_merch_name,auth_amt,local_amt,cleaned_mname_1,mname,state,city
75220,2021-08-29 08:20:50.000,7999,188418000053360,SQ *PG POOL Mount Rainier MD,1.00,1.00,sq pg pool mount rainier md,sq pg pool mount rainier,md,
48955,2021-08-14 01:41:27.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,12.83,12.83,apple ca,apple,ca,
44966,2021-08-14 01:45:28.000,5818,342475000144509,SIE*PLAYSTATIONNETWORK 877-971-7669 CA,4.97,4.97,sie playstationnetwork ca,sie playstationnetwork,ca,
13568,2021-08-03 06:14:58.000,5815,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.00,0.00,apple ca,apple,ca,
92727,2021-08-29 08:14:41.000,5735,311204598883,APPLE.COM/BILL www.apple.com CA,9.99,9.99,apple apple com ca,apple apple com,ca,
51349,2021-09-09 08:48:58.000,5942,784959000762203,Amazon.com Amzn.com/bill WA,2.02,2.02,amazon com amzn wa,amazon com amzn,wa,
86979,2021-08-15 02:43:03.000,5942,784959000762203,AMAZON.COM AMZN.COM/BILL WA,0.00,0.00,amazon com amzn wa,amazon com amzn,wa,
3806,2021-09-09 13:31:35.000,5735,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.99,0.99,apple ca,apple,ca,
91822,2021-09-09 06:57:33.000,5735,112137000108778,APPLE.COM/BILL 866-712-7753 CA,0.00,0.00,apple ca,apple,ca,
6006,2021-08-03 05:38:48.000,5816,145376000144509,PLAYSTATION NETWORK 800-345-7669 CA,7.99,7.99,playstation network ca,playstation network,ca,


In [138]:
df_train[df_train['city']!= ''].head(10)

,auth_ts,mcc,mid,auth_merch_name,auth_amt,local_amt,cleaned_mname_1,mname,state,city
55086,2021-08-29 17:49:12.000,7832,000372089626884,REG LOUISIANA 14 0664 BOSSIER CITY LA,7.10,7.10,reg louisiana bossier city la,reg louisiana,la,bossier city
44204,2021-09-09 09:01:28.000,5814,679313000000000,MCDONALD'S F3955 INDIANAPOLIS IN,8.79,8.79,mcdonald s indianapolis in,mcdonald s,in,indianapolis
76308,2021-09-09 08:59:40.000,5814,321557000000000,DUNKIN #330016 Q35 STROUDSBURG PA,7.42,7.42,dunkin stroudsburg pa,dunkin,pa,stroudsburg
29365,2021-08-29 08:31:08.000,5814,321028000000000,DUNKIN #306582 Q35 BRIGHTON MA,1.38,1.38,dunkin brighton ma,dunkin,ma,brighton
74932,2021-08-10 09:36:35.000,5542,57542757406,SHELL UNION NJ,15.00,15.00,shell union nj,shell,nj,union
76079,2021-08-10 15:39:39.000,5812,542929806582128,CHICKEN EXPRESS - MANS ARLINGTON TX,8.10,8.10,chicken express mans arlington tx,chicken express mans,tx,arlington
59926,2021-08-29 17:17:49.000,5814,000335207524885,CINNABON #WA003 KENNEWICK WA,15.60,15.60,cinnabon kennewick wa,cinnabon,wa,kennewick
41075,2021-08-10 04:14:44.000,5817,527021000211443,GOOGLE* ROBLOX Mountain View CA,1.07,1.07,google roblox mountain view ca,google roblox,ca,mountain view
57045,2021-08-10 15:32:18.000,5422,000000011578922,WILD COUNTRY MEATS CLEVELAND OK,5.48,5.48,wild country meats cleveland ok,wild country meats,ok,cleveland
80164,2021-09-09 11:27:35.000,5818,527021000222747,PlaystationNetwork San Mateo CA,19.99,19.99,playstationnetwork san mateo ca,playstationnetwork,ca,san mateo


In [139]:
df_train.to_csv('../data/df_train_raw.csv', index=False)
